In [3]:
import json
import pprint
import numpy
import websocket
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
import numpy as np
import pickle
from binance.client import Client

In [4]:
#load trained model if needed
loaded_model = pickle.load(open('binance_model.pkl', 'rb'))

In [6]:
#create binance client
client = Client('***', '***')
balance = client.get_asset_balance(asset='USDT')

In [7]:
#check balance
balance["free"]

'10.64586701'

In [8]:
#load historic data
data = pd.read_csv("Binance_BTCUSDT_minute.csv", skiprows=[0])

In [9]:
len(data)

304998

In [10]:
#leave only time, open, high, low, close
df = data.drop(["date", "symbol","Volume BTC", "Volume USDT", "tradecount"], axis=1)

In [11]:
df.head(120)

,unix,open,high,low,close
0,1618187760000,59720.22,59730.41,59719.99,59730.15
1,1618187700000,59724.99,59734.80,59716.19,59720.21
2,1618187640000,59704.07,59729.27,59704.06,59724.99
3,1618187580000,59660.47,59708.86,59658.23,59704.07
4,1618187520000,59701.04,59701.05,59660.47,59660.48
...,...,...,...,...,...
115,1618180860000,59849.39,59885.07,59843.50,59850.01
116,1618180800000,59773.35,59853.92,59769.86,59849.40
117,1618180740000,59792.01,59800.00,59763.62,59773.35
118,1618180680000,59803.91,59839.32,59789.51,59792.02


In [13]:
# create df with closed minute values for 2 hours and value an hour later
hour_data = pd.DataFrame(columns=[*range(1,121)])
hour_data["end"] = 0
for i in range(1,10000):
  values = list(df['open'][60+30*i:180+30*i])
  values.append(df['close'][30*i])
  hour_data.loc[len(hour_data)] = values

In [14]:
 hour_data

,1,2,3,4,5,6,7,8,9,10,...,112,113,114,115,116,117,118,119,120,end
0,60006.08,60030.42,60039.71,59989.27,60023.41,60059.20,60101.68,60037.12,59978.31,59989.08,...,59603.01,59629.45,59638.87,59599.72,59587.94,59528.26,59563.93,59569.01,59592.83,60094.65
1,59878.46,59890.59,59880.34,59934.30,60041.82,60079.99,60009.72,59987.79,60055.66,60017.55,...,59629.30,59624.89,59635.64,59650.02,59637.63,59638.27,59630.01,59661.54,59709.80,59893.92
2,59460.27,59456.39,59500.00,59542.51,59549.30,59527.56,59610.72,59579.09,59568.10,59577.15,...,59669.03,59730.27,59671.67,59700.02,59730.68,59715.95,59729.24,59722.88,59680.03,59984.61
3,59584.21,59601.45,59519.01,59498.29,59506.13,59502.79,59542.72,59594.24,59584.58,59548.24,...,59711.93,59785.48,59750.31,59766.07,59798.56,59778.08,59747.14,59679.48,59653.66,59830.45
4,59598.47,59611.52,59605.43,59625.33,59627.57,59593.10,59707.79,59643.79,59624.65,59579.29,...,60011.50,60009.70,60000.29,60016.04,60006.97,60001.79,60000.00,59949.98,59926.68,59488.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,10653.61,10648.96,10652.27,10658.01,10660.00,10657.85,10662.81,10665.79,10661.73,10663.50,...,10741.80,10744.38,10747.38,10745.74,10753.86,10751.06,10757.26,10737.36,10734.01,10685.99
9995,10655.63,10663.12,10649.87,10638.45,10632.99,10638.65,10645.10,10643.54,10635.26,10647.04,...,10721.52,10721.12,10720.39,10721.00,10718.23,10726.20,10736.05,10727.00,10724.06,10696.68
9996,10718.77,10719.54,10717.00,10716.00,10719.33,10724.10,10721.66,10724.26,10722.24,10717.71,...,10706.61,10705.61,10709.26,10711.45,10713.49,10714.55,10718.64,10725.28,10726.63,10652.88
9997,10738.92,10736.65,10735.85,10734.92,10730.20,10730.53,10727.25,10729.17,10728.97,10729.27,...,10718.46,10718.65,10719.16,10724.41,10725.76,10731.09,10742.04,10741.82,10743.45,10655.62


In [11]:
#prepare data
X = hour_data.drop('end', axis=1)
y = hour_data['end']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
#try different estimators
reg1 = GradientBoostingRegressor(n_estimators=100)
reg2 = RandomForestRegressor()
reg3 = LinearRegression()
reg4 = linear_model.Ridge(alpha=.5)
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3),('ri',reg4)])

ereg.fit(X_train, y_train)

VotingRegressor(estimators=[('gb',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                    

In [14]:
#check if they are right on value going up 
pred_ereg = ereg.predict(X_test[:])
pred_load = loaded_model.predict(X_test[:])
last = X_test.iloc[:,-1]
real = np.array(y_test)
last = np.array(last)
#find the best estimator
check = pd.DataFrame([last, pred_ereg, pred_load, real], index=['last','pred_ereg','pred_load','real']).transpose()
check.loc[((check['real']> check['last']) & (check['pred_load'] > check['last']))+((check['real'] < check['last']) & (check['pred_load'] < check['last']))]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  warnings.warn(


,last,pred_ereg,pred_load,real
0,10996.78,10960.189604,10962.192971,10980.57
1,44315.15,45188.840689,45211.869444,45281.07
2,57245.01,59522.847144,59598.200278,60056.34
3,19058.09,19084.381807,19087.411675,19119.41
4,47719.96,47754.902580,47709.962748,47437.20
...,...,...,...,...
1995,22939.39,22964.829924,22986.381232,23061.75
1996,59289.28,58859.275178,58838.410870,58740.42
1997,22839.60,22978.436700,22990.347930,22980.93
1998,31729.66,31243.527735,31120.333959,31059.02


In [429]:
#save model
pkl_filename = "binance_model_saved.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(ereg, file)

In [430]:
#minute closes
closes = []

In [431]:
#buy time, bought prices and values that we bought more than an hour before
time = []
have = []
expired = []

In [ ]:
# connect to site
SOCKET = "wss://stream.binance.com:9443/ws/btcusdt@kline_1m"




def on_open(ws):
    print('opened connection')


def on_close(ws):
    print('closed connection')

# every time btc value closes
def on_message(ws, message):
    global closes, i    

    json_message = json.loads(message)

    candle = json_message['k']
    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        closes.append(float(close))
        #if more than 2 hours passed after we started program
        if len(closes)>120:
            last_hours = np.array(closes[-120:]) 
            #predict value of last 2 hours and compare with the last one
            if closes[-1]<float(loaded_model.predict(last_hours.reshape(1,-1))):
                #buy btc if pred val > last val 
                balance_usd = client.get_asset_balance(asset='USDT')
                if float(balance_usd['free'])>16:
                    order = client.create_order(
                        symbol='BTCUSDT',
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity = 0.0003)   
                    #add bought time and at what price we bought
                    time.append(int(candle['t'])+3600000)
                    print("bought for {}".format(float(closes[-1])))
                    have.append(float(closes[-1]))
        #if bought time was more than 2 hours ago than add bought val to expired     
        expired = have[:sum(i<candle['t'] for i in time)]
        if len(expired)>0:
            #if val in expired < than value now then sell
            if min(expired)<closes[-1]:                
                    balance_btc = client.get_asset_balance(asset='BTC')
                    if float(balance_btc['free'])>0.0003:
                        print('sold for {}'.format(closes[-1]))
                        order = client.create_order(
                            symbol='BTCUSDT',
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity = 0.0003) 
                        #del sold value
                        del time[have.index(min(expired))]
                        have.remove(min(expired))                    
                        expired.remove(min(expired))
                    
                    


ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

opened connection
bought for 51300.0
bought for 51140.69
bought for 51176.64
